In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, 
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# importing Random Forest classifier
from sklearn.ensemble import RandomForestClassifier


import os
from pathlib import Path
import time
from collections import OrderedDict, namedtuple

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=57ce0227e6d8d28b2cd931de5e277a92ead6be37162376d02102f6aceadfe641
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
if IN_COLAB:

    from google.colab import drive
    drive._mount('/drive', force_remount=True)
    datadir = '/drive/My Drive/Colab Notebooks/Datasets/'
    optim_dir = datadir + '/optim_dir/'
    results_folder = datadir+'/results/UCI/'
    Path(results_folder).mkdir(parents=True, exist_ok=True)
    Path(optim_dir).mkdir(parents=True, exist_ok=True)
else:
  results_folder = 'results'
  optim_dir = 'optim_dir'
  Path("results").mkdir(parents=True, exist_ok=True)
  Path(optim_dir).mkdir(parents=True, exist_ok=True)


# from opfunu.cec_basic.cec2014_nobias import *
# from mealpy.swarm_based.PSO import BasePSO

Mounted at /drive


In [ ]:
import sys

!pip install category_encoders
import category_encoders as ce

if IN_COLAB:
  
  scripts_dir = '/drive/My Drive/Colab Notebooks/scripts/'
  sys.path.insert(1, scripts_dir)
# from opfunu.cec_basic.cec2014_nobias import *
# from mealpy.swarm_based.PSO import BasePSO

# insert at 1, 0 is the script path (or '' in REPL)
else:
    sys.path.insert(1, 'scripts')
from bayes_opt import BayesianOptimization
import pickle

     |████████████████████████████████| 82 kB 320 kB/s 


In [ ]:
# Enable GPU processing
# if IN_COLAB:
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# else:
#     device = torch.device("cpu")
# print(f'Device type: {device}')

In [ ]:
# Set random seed
seed = 777
# torch.manual_seed(seed)
model = None

In [ ]:
import random
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    os.environ['TF_CUDNN_DETERMINISTIC'] = str(SEED)
    # tf.random.set_seed(SEED)
    print('seeding done!!!')
seeding(seed)

seeding done!!!


In [ ]:
df = pd.read_csv(f"{datadir}UCI/Letter/data.csv")

In [ ]:
df.head()

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [ ]:
df.shape

(20000, 17)

In [ ]:
x = df.drop(['letter'], axis=1)
y = df['letter']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42) # Split dataset into train and test sets

In [ ]:
X_train.shape, X_test.shape # Get the dimensions of features

((16000, 16), (4000, 16))

In [ ]:
X_train.head()

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
5894,4,7,5,5,4,6,7,3,7,11,8,9,3,8,4,8
3728,4,7,6,5,5,6,8,3,7,11,8,9,3,8,4,7
8958,3,5,4,3,3,7,8,5,5,7,7,6,5,9,2,6
7671,4,10,6,7,6,5,7,5,7,6,6,12,3,8,6,9
5999,4,10,6,8,4,8,11,2,3,4,10,9,3,11,1,8


In [ ]:
# Create a class to manage the training / hyperparameter runs
class RunManager():
  def __init__(self):
    self.run_params = None
    self.run_count = 0
    self.run_data = []
    
    #---
    self.results = None

  def begin_run(self, run):
    self.run_params = run
    self.run_count += 1
    

  def end_run(self, train_accuracy, test_accuracy):
    # self.epoch_count = 0
    # train_accuracy = 
    # test_accuracy = 

    print(f'run_params: {self.run_params} Train Acc: {train_accuracy} Test Acc: {test_accuracy}')

    results = OrderedDict()
    results['run'] = self.run_count
    results['train acc'] = train_accuracy
    results['test acc'] = test_accuracy
    
    # ---
    self.results = results

    for k, v in self.run_params.items():
      results[k] = v

    self.run_data.append(results)

  def save_output(self, filename):
    if filename:
      filename = filename
      pd.DataFrame.from_dict(self.run_data, orient='columns').to_csv(f'{filename}.csv')
      
      # with open(f'{filename}.json', 'w', encoding='utf-8') as f:
      #   json.dump(self.run_data, f, ensure_ascii=False, indent=4)

      # print('Results saved to disk')

    return pd.DataFrame.from_dict(self.run_data, orient='columns')


In [ ]:
# Define training loop execution
def execution_loop(m, args): # args is given by the optimizer
    agent = {}
    agents = []
    
    agent['n'] = args[0].astype(int)
    agent['m'] = int(args[1])
    agents.append(agent)
#     m = RunManager()
    for run in agents: # this should be one. a particle in pso

        # print(f'Run Params: {run}')

        m.begin_run(run)

        num_predictors = X_train.shape[1]
        if run['m'] <= num_predictors:

          rfc = RandomForestClassifier(n_estimators=run['n'], max_features=run['m'], random_state=0)
          rfc.fit(X_train, y_train)

          y_pred_train = rfc.predict(X_train)
          y_pred_test = rfc.predict(X_test)

          train_score = accuracy_score(y_train, y_pred_train)
          test_score = accuracy_score(y_test, y_pred_test)
        else:
          train_score = 0
          test_score = 0

        m.end_run(train_score, test_score)
    return m.results['test acc']

In [ ]:
# Define training run hyperparameters
params = {
    'n' : [1, 200],
    'm' : [1, 20]
}

# params = OrderedDict(
#     lr = [0, 1],
#     batch_size = [20 2000],
#     n_epochs = [3]
# )

In [ ]:
m = RunManager()
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = f'Run_Results-UCI-Letter-BO-{timestr}'
# filename = 'Run_Results-PSO-20210504-035642-cont.csv' # this is for trying to continue from last optim save


In [ ]:
def run_train_model(agent):
    test_acc = execution_loop(m, agent)
    m.save_output(f'{results_folder}/{filename}')
    print("test acc: ", test_acc)
    return test_acc * -1

In [ ]:
def run_train_model_proxy(n_estimators, max_features):
    agent = [n_estimators, max_features]
    print(agent)
    return run_train_model(agent) * -1

In [ ]:
verbose = True
pop_size = 15   
max_iter = 10

obj_func = run_train_model_proxy

lb = [params['n'][0], params['m'][0]]
ub = [params['n'][1], params['m'][1]]

tic = time.perf_counter()
# Run Bayesian Optimization

params_gbm ={
    'max_features':(1, 20), 'n_estimators':(1, 200)
}

gbm_bo = BayesianOptimization(f = obj_func, pbounds = params_gbm, random_state=777)
gbm_bo.maximize(init_points=15, n_iter=135)

toc = time.perf_counter()
print(f'Time to execute: {toc - tic:0.4f} seconds')
# print(f'Best Position: {[int(x) for x in best_pos1]} Best Solution: {best_fit1:0.5f}')
print(gbm_bo.max)


# m.save_output(f'{results_folder}/{filename}')
# print('Results saved.')


|   iter    |  target   | max_fe... | n_esti... |
-------------------------------------------------
[61.16896533179672, 3.9006109631251182]
run_params: {'n': 61, 'm': 3} Train Acc: 1.0 Test Acc: 0.95975
test acc:  0.95975
|  1        |  0.9597   |  3.901    |  61.17    |
[92.5122079801848, 2.178691879576678]
run_params: {'n': 92, 'm': 2} Train Acc: 1.0 Test Acc: 0.96
test acc:  0.96
|  2        |  0.96     |  2.179    |  92.51    |
[185.47241250222905, 16.869814304314936]
run_params: {'n': 185, 'm': 16} Train Acc: 1.0 Test Acc: 0.938
test acc:  0.938
|  3        |  0.938    |  16.87    |  185.5    |
[153.93074816220832, 14.812790702062332]
run_params: {'n': 153, 'm': 14} Train Acc: 1.0 Test Acc: 0.94525
test acc:  0.94525
|  4        |  0.9453   |  14.81    |  153.9    |
[129.16182912212687, 6.114896262971671]
run_params: {'n': 129, 'm': 6} Train Acc: 1.0 Test Acc: 0.96175
test acc:  0.96175
|  5        |  0.9617   |  6.115    |  129.2    |
[16.857491236685103, 2.774091879755339]
run_p